#### 标签分群监控==》极简测试版

In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import time
import sys
import requests
import json
import pymysql
import time
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

print
"标签分群检测预警"

# print "脚本位置：", sys.argv[0]
# print "任务参数：", sys.argv[1]
# print "分片序号：", sys.argv[2]
# print "分片总数：", sys.argv[3]

# MySQL数据库连接信息
HOST = '10.88.10.72'
USER = 'root'
PASSWORD = 'YdaD6n9dBjF$l&D3'
DB = 'bigdata_general_tag'
PORT = 3306

# 上次状态记录
last_state = {}

WEBHOOK_URL = 'https://qyapi.weixin.qq.com/cgi-bin/webhook/send?key=372def7c-1657-4ee3-bf69-0d3b44581d14'


def get_db_connection():
    return pymysql.connect(
        host=HOST,
        user=USER,
        password=PASSWORD,
        db=DB,
        port=PORT,
        charset='utf8mb4'
    )


def send_wechat_message(content):
    message = {
        "msgtype": "text",  # 消息类型
        "at": {
            "isAtAll": "true"
        },
        "text": {
            "content": content  # 发送的消息内容
        }
    }

    try:
        response = requests.post(WEBHOOK_URL, data=json.dumps(message), headers={'Content-Type': 'application/json'})
        if response.status_code == 200:
            print("消息发送成功")
        else:
            # print(f"发送失败，状态码: {response.status_code}, 错误信息: {response.text}")
            print
            "发送失败，状态码: %s, 错误信息: %s" % (response.status_code, response.text)
    except Exception as e:
        print
        "发生错误:", e


# 监控并报警
def monitor_tag_status():
    global last_state
    con = get_db_connection()
    msg="";
    try:
        with con.cursor() as cursor:
            # 状态检测：
            query = f'''
            SELECT
                common_tag_id,
                name,
                common_tag_type,
                in_refresh,
                TIMESTAMPDIFF(HOUR,compute_start, NOW()) AS delay
            FROM common_tag
            WHERE (in_refresh = 1 OR in_refresh = 2)
              AND compute_start < NOW()
              AND tag_cate ='自动' OR tag_cate is null
            -- #   AND compute_end < NOW()
            HAVING delay > 1
            '''
            cursor.execute(query)
            results = cursor.fetchall()
            for row in results:
                common_tag_id, name, common_tag_type,in_refresh,delay = row
                name = name.encode('utf-8')
                if in_refresh == 1: # 1排队中 2计算中
                    if common_tag_type == 'tag': #标签
                        TAG = '标签'
                    if common_tag_type == 'cluster': #分群
                        TAG = '分群'
                    msg = msg + "\n TAGID[%s] , %s %s 排队时长已超1小时" % (common_tag_id, TAG,name.decode('utf-8'))
                if in_refresh == 2: # 1排队中 2计算中
                    if common_tag_type == 'tag': #标签
                        TAG = '标签'
                    if common_tag_type == 'cluster': #分群
                        TAG = '分群'
                    msg = msg + "\n TAGID[%s] , %s: %s 计算时长已超1小时" % (common_tag_id, TAG,name.decode('utf-8'))

            if msg:
                msg = "标签分群告警:" + msg
                # send_wechat_message(msg)
                print(f"{msg}")
    finally:
        con.close()

monitor_tag_status();

print
"Task Finish!"
exit(0)

标签分群告警:
 TAGID[7fb42b8c-54b8-4c83-a0ea-39a6dda5705b] , 分群: 自动化-test-001 计算时长已超1小时
 TAGID[5bb325aa-0002-4eab-8775-9b07fe8dbff2] , 分群: 自动化-test-001 计算时长已超1小时
 TAGID[7e215f1a-e93a-42aa-8f2b-6eb58ed056fb] , 分群 自动化-test-001 排队时长已超1小时
 TAGID[fdc28cda-7bdc-438b-a388-acbb74c1fb2b] , 分群: 用户库评级 计算时长已超1小时
 TAGID[16c5ce8b-7743-46b7-9ae2-e12a67b4e088] , 标签: 画布周期性删除标签4 计算时长已超1小时


### 标签分群监控

In [2]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import time
import sys
import requests
import json
import pymysql
import time
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

print
"标签分群检测预警"

# print "脚本位置：", sys.argv[0]
# print "任务参数：", sys.argv[1]
# print "分片序号：", sys.argv[2]
# print "分片总数：", sys.argv[3]

# MySQL数据库连接信息
HOST = '10.88.10.72'
USER = 'root'
PASSWORD = 'YdaD6n9dBjF$l&D3'
DB = 'bigdata_general_tag'
PORT = 3306

# 上次状态记录
last_state = {}

WEBHOOK_URL = 'https://qyapi.weixin.qq.com/cgi-bin/webhook/send?key=372def7c-1657-4ee3-bf69-0d3b44581d14'


def get_db_connection():
    return pymysql.connect(
        host=HOST,
        user=USER,
        password=PASSWORD,
        db=DB,
        port=PORT,
        charset='utf8mb4'
    )


def send_wechat_message(content):
    message = {
        "msgtype": "text",  # 消息类型
        "at": {
            "isAtAll": "true"
        },
        "text": {
            "content": content  # 发送的消息内容
        }
    }

    try:
        response = requests.post(WEBHOOK_URL, data=json.dumps(message), headers={'Content-Type': 'application/json'})
        if response.status_code == 200:
            print("消息发送成功")
        else:
            print
            "发送失败，状态码: %s, 错误信息: %s" % (response.status_code, response.text)
    except Exception as e:
        print
        "发生错误:", e


# 监控并报警
def monitor_tag_status():
    global last_state
    con = get_db_connection()
    msg="";
    try:
        with con.cursor() as cursor:
            # 状态检测：
            query = f'''
            SELECT
                common_tag_id,
                name,
                common_tag_type,
                in_refresh,
                TIMESTAMPDIFF(HOUR,
                              (SELECT compute_start
                               FROM common_tag
                               WHERE compute_start < NOW()
                                 AND compute_end < NOW()
                                 AND common_tag_category_id IN (
                                     SELECT common_tag_category_id
                                     FROM bigdata_general_tag.common_tag_category ctc
                                     WHERE target_entity_api_name = 'xiaokuihua'
                                       AND tenant_api_name = 't100000'
                                 )
                              ), NOW()) AS delay
            FROM common_tag
            WHERE (in_refresh = 1 OR in_refresh = 2)
              AND compute_start < NOW()
              AND compute_end < NOW()
              AND tag_cate ='自动' OR tag_cate is null
              AND common_tag_category_id IN (
                  SELECT common_tag_category_id
                  FROM bigdata_general_tag.common_tag_category ctc
                  WHERE target_entity_api_name = 'xiaokuihua'
                    AND tenant_api_name = 't100000'
              )
            HAVING delay > 1;
            '''
            cursor.execute(query)
            results = cursor.fetchall()
            for row in results:
                common_tag_id, name, common_tag_type,in_refresh,delay = row
                name = name.encode('utf-8')
                if in_refresh == 1: # 1排队中 2计算中
                    if common_tag_type == 'tag': #标签
                        TAG = '标签'
                    if common_tag_type == 'cluster': #分群
                        TAG = '分群'
                    msg = msg + "\n TAGID[%s] , %s %s 排队时长已超1小时" % (common_tag_id, TAG,name.decode('utf-8'))
                if in_refresh == 2: # 1排队中 2计算中
                    if common_tag_type == 'tag': #标签
                        TAG = '标签'
                    if common_tag_type == 'cluster': #分群
                        TAG = '分群'
                    msg = msg + "\n TAGID[%s] , %s: %s 计算时长已超1小时" % (common_tag_id, TAG,name.decode('utf-8'))

            if msg:
                msg = "标签分群告警:" + msg
                send_wechat_message(msg)
                # print(f"{msg}")
    finally:
        con.close()

monitor_tag_status();

print
"Task Finish!"
exit(0)

In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import time
import sys
import requests
import json
import psycopg2
import time
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

print
"MA监控"

# print "脚本位置：", sys.argv[0]
# print "任务参数：", sys.argv[1]
# print "分片序号：", sys.argv[2]
# print "分片总数：", sys.argv[3]

# MySQL数据库连接信息
HOST = '10.88.10.73'
USER = 'postgres'
PASSWORD = 'R4BuypN9rgeUt7qk'
DB = 'bigdata_flow'
PORT = 5432

# 上次状态记录
last_state = {}


WEBHOOK_URL = 'https://qyapi.weixin.qq.com/cgi-bin/webhook/send?key=372def7c-1657-4ee3-bf69-0d3b44581d14'


def get_db_connection():
    return psycopg2.connect(
        host=HOST,
        user=USER,
        password=PASSWORD,
        database=DB,
        port=PORT
    )


def send_wechat_message(content):
    message = {
        "msgtype": "text",  # 消息类型
        "at": {
            "isAtAll": "true"
        },
        "text": {
            "content": content  # 发送的消息内容
        }
    }

    try:
        response = requests.post(WEBHOOK_URL, data=json.dumps(message), headers={'Content-Type': 'application/json'})
        if response.status_code == 200:
            print("消息发送成功")
        else:
            print
            "发送失败，状态码: %s, 错误信息: %s" % (response.status_code, response.text)
    except Exception as e:
        print
        "发生错误:", e


# 监控并报警
def monitor_ma_status():
    global last_state
    con = get_db_connection()
    msg="";
    try:
        with con.cursor() as cursor:
            # 状态检测：
            query = f'''
            select flow_id, flow_name
            from (select fi.flow_id,
                         fi.flow_name,
                         fi.flow_status,
                         SUM(CASE "node_key" WHEN 'start' THEN total ELSE 0 END) as stt,
                         SUM(CASE "node_key" WHEN 'end' THEN total ELSE 0 END)   as ed
                  from public.flow_info fi
                           left join nodedata.flow_data_log_agg fla on fi.flow_id = fla.flow_id
                  where fi.flow_id in (select flow_id
                                       from flow_info fi
                                       where flow_status = 3)
                  group by fi.flow_id, fi.flow_name, fi.flow_status) a
            where stt <> ed;
            '''
            cursor.execute(query)
            results = cursor.fetchall()
            for row in results:
                flow_id, flow_name= row
                flow_name = flow_name.encode('utf-8')
                status = "进度未达100%"
                # 检查是否需要发出警报
                msg = msg + "\n MAID[%s] , %s 执行结束，%s" % (flow_id, flow_name.decode('utf-8'), status)

            if msg:
                msg = "MA监控告警:" + msg
                send_wechat_message(msg)
                # print(f"{msg}")
    finally:
        con.close()

monitor_ma_status();

print
"Task Finish!"
exit(0)


MA监控告警:
 MAID[3762] , 日期类型-包含以...结尾 执行结束，进度未达100%
 MAID[815] , 测试1_复制1 执行结束，进度未达100%
 MAID[2054] , 行为判断-时间-等于_下周_复制 执行结束，进度未达100%
 MAID[5604] , 等待111 执行结束，进度未达100%
 MAID[4634] , 属性指标浮点型_大于等于不进人 执行结束，进度未达100%
 MAID[1967] , 属性判断-主数据-时间-日-等于_复制 执行结束，进度未达100%
 MAID[3755] , 日期类型-包含 执行结束，进度未达100%
 MAID[5581] , 等待插件_复制_复制 执行结束，进度未达100%
 MAID[2675] , 刚下单客户_关注公众号_平平 执行结束，进度未达100%
 MAID[2925] , 属性多分支周期性 执行结束，进度未达100%
 MAID[4429] , 复测90804 执行结束，进度未达100%
 MAID[3107] , 属性多分支模板套用1 执行结束，进度未达100%
 MAID[936] , 实er222时行为判断__复制123 执行结束，进度未达100%
 MAID[761] , 实时行为判断 执行结束，进度未达100%
 MAID[5851] , 会员拉新 执行结束，进度未达100%
 MAID[2954] , 属性多分支周期性_复制2 执行结束，进度未达100%
 MAID[5322] , 接口监控自动化测试流程_rengong 执行结束，进度未达100%
 MAID[5708] , 98985uu 执行结束，进度未达100%
 MAID[3780] , 日期类型-包含以...结尾为空 执行结束，进度未达100%
 MAID[4254] , 指标_绝对时间_验证_复制 执行结束，进度未达100%
 MAID[4029] , 行为多分支_历史行为_日期_平平1 执行结束，进度未达100%
 MAID[6005] , 画布12 执行结束，进度未达100%
 MAID[2688] , 提交表单_关注公众号_平平 执行结束，进度未达100%
 MAID[3000] , 提交表单_关注公众号_平平_复制 执行结束，进度未达100%
 MAID[3019] , 属性多分支整型小于等